In [44]:
# Bugs to fix:


# cpu to gpu
# why is loss rising?
# Attention layer


In [1]:
import os
import torch as t
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

from itertools import repeat
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import optim

device = t.device("cpu")
# device = t.device("mps")
# device = t.device("mps" if t.backends.mps.is_available() else "cpu")

In [2]:
# Load in Training Files

directory = "corpus_files"

en_corpus_file = "train_en.txt"
en_file = open(directory + "/" + en_corpus_file, "r", encoding="utf-8")
en_sentences = en_file.read()
en_file.close()
en_sentences = en_sentences.split('\n')
# print(en_sentences[0])

gu_corpus_file = "train_gu.txt"
gu_file = open(directory + "/" + gu_corpus_file, "r", encoding="utf-8")
gu_sentences = gu_file.read()
gu_file.close()
gu_sentences = gu_sentences.split('\n')
# print(gu_sentences[0])

In [3]:
# How many English and Gujarati sentences? Confirm they're the same amount
print(len(en_sentences))
print(len(gu_sentences))

65001
65001


In [4]:
# Load in Vocab Files

en_tokens_file = "vocab_en.txt"
en_file = open(directory + "/" + en_tokens_file, "r")
en_words = en_file.read()
en_file.close()
en_words =  en_words.split('\n')

# strip leading spaces from the word strings
# add the pad token
en_words.append("</pad>")
print(max(en_words))

gu_tokens_file = "vocab_gu.txt"
gu_file = open(directory + "/" + gu_tokens_file, "r")
gu_words = gu_file.read()
gu_file.close()
gu_words =  gu_words.split('\n')

# get rid of the zero-byte characters
gu_words = [word.replace("\u200b","") for word in gu_words]
# strip leading spaces from the word strings
gu_words.append("</પેડ>")
print(max(gu_words))


zucchini.
હ્યુસ્ટનમાં


In [5]:
# Tokenize

# build a dictionary
# word to index for both langs

eng_word2index = {}
eng_index2word = {}
for i, word in enumerate(en_words):
    eng_word2index[word] = i
    eng_index2word[i] = word

guj_word2index = {}
guj_index2word = {}
for i, word in enumerate(gu_words):
    guj_word2index[word] = i
    guj_index2word[i] = word

In [7]:
# Get collection of tokenized indexes for each string

# english [[1 sentence words tokenized], []]
# guj [[1 sentence words tokenized], []]

# english word to index
max_eng_sent_len = 55 # this is the buffer I set for the longest sent length. Based on the longest sentence before adding this in
eng_seq_collection = []
for sentence in en_sentences:
    eng_seq_sentence = []
    # Start and end each sequence with <s> and </s>
    # This allows the model to know when each string stops and starts (when it should stop predicting the next token)
    eng_seq_sentence.append(eng_word2index['<s>'])
    for word in sentence.strip().split(' '):
        eng_seq_sentence.append(eng_word2index[word])

    eng_seq_sentence.append(eng_word2index['</s>'])
    # # determine the max sentence length
    # max_eng_sent_len = max(max_eng_sent_len, len(eng_seq_sentence))
    # pad sentences to all be the same length
    difference = max_eng_sent_len - len(eng_seq_sentence)
    if difference > 0:
        eng_seq_sentence.extend(repeat(eng_word2index["</pad>"], difference))
        
    eng_seq_collection.append(eng_seq_sentence)
    

# gujarati word to index
max_guj_sent_len = 50 # this is the buffer I set for the longest sent length. Based on the longest sentence length before adding this in
guj_seq_collection = []
for sentence in gu_sentences:
    guj_seq_sentence = []
    guj_seq_sentence.append(guj_word2index['<s>'])
    for word in sentence.strip().split(' '):
        # get rid of the zero-byte characters
        try:
            word = word.replace("\u200b","")
            guj_seq_sentence.append(guj_word2index[word])
        except KeyError:
            print("not working", sentence)
            raise KeyError
    
    guj_seq_sentence.append(guj_word2index['</s>'])

    # pad sentences to all be the same length
    difference = max_guj_sent_len - len(guj_seq_sentence)
    if difference > 0:
        guj_seq_sentence.extend(repeat(guj_word2index["</પેડ>"], difference))
    # # determine the max sentence length
    # max_guj_sent_len = max(max_guj_sent_len, len(guj_seq_sentence))

    guj_seq_collection.append(guj_seq_sentence)


    # list of indexes for each sentence

In [8]:
# confirm that all the sentences are the same size
for sentence in guj_seq_collection[0:5]:
    print(len(sentence))

for sentence in eng_seq_collection[0:5]:
    print(len(sentence))

50
50
50
50
50
55
55
55
55
55


In [9]:
# This is code that I ran BEFORE adding in the sentences padding

len_of_sent = []
for i in guj_seq_collection:
    len_of_sent.append(len(i))
# Find longest sentence
print(max(len_of_sent))

# max seq len = 50


len_of_sent = []
for i in eng_seq_collection:
    len_of_sent.append(len(i))
# Find longest sentence
print(max(len_of_sent))

# max seq len = 55

50
55


In [10]:
# input tensor for embedding
# [batch size, token length]
# vocab size: 17472 (english)
# token length (arbitarry) - 50 or 60 tokens

# dummy encoder input
batch_size = 10 # how should this be determined?
eng_vocab_size = 17500 # based on vocab size above
sequence_length = 56 # based on max sentence length (english sentencelength)

encoder_input = t.tensor(t.zeros(batch_size, sequence_length)).long()

# decoder output: [batch, seqlen, vocab]
# to [batch, seqlen] (using argmax)
# convert output to list: .list()
# use dict to convert from indexes to words

print(encoder_input[0])
print(encoder_input[0].shape)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([56])


/var/folders/ww/cqrdw531507g4cfv8ncd8fz80000gp/T/ipykernel_1216/2268196282.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encoder_input = t.tensor(t.zeros(batch_size, sequence_length)).long()


In [11]:
class EncoderLSTM(nn.Module):
    def __init__(self, embedding_dim, eng_vocab_size, hidden_size, num_layers, dropout_p=0.1):
        """
        embedding_dim: for each word in vocab_size, this is the number of floats representing each word
        vocab_size: the size of the original english vocab len(en_words)
        hidden_size: the number of LSTM cells in each layer
        num_layers: the number of LSTM layers in the model
        dropout_p: 
        """
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        # Step 1: Create the Embedding Matrix
            # input dimensions: tensor[batch_size, sequence_length]
            # output dimensions: tensor[batch_size, sentence_length, embedding_dim]
            # creates the embedding matrix to be of dimensions vocab size x embedding_dim
            # the actual values are random to begin with
            # the embedding matrix is unique (lookup table, key is the index, value is the embedding for that word)
        self.embedding = nn.Embedding(eng_vocab_size, embedding_dim)
        # this just builds the LSTM architechture. Also random floats
        # Step 2: Run the LSTM Model
            # In: input, (h_n, c_n)
            # input dimensions: (for batched) tensor[batch_size, sequence_length, input_size] AKA tensor[batch_size, sentence_length, embedding_dim]
            # h_n input dimensions: tensor[#_layers, hidden_size] of zeros since we don't specify
            # h_c input dimensions: tensor[#_layers, batch_size, hidden_size] of zeros since we don't specify
            # Out: output, (h_n, c_n)
            # output dimensions: (for batched) tensor[batch_size, sequence_length, hidden_size] AKA tensor[batch_size, sentence_length, number_of_LSTMcells]
            # h_n output dimensions: this is the hidden output (short term memory). tensor[#_layers, batch_size, hidden_size]
            # h_c output dimensions: this is the cell output (long term memory). tensor[#_layers, batch_size, hidden_size]
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, dropout=dropout_p, batch_first=True)

    def forward(self, input):
        embedded = self.embedding(input)
        output, hidden = self.lstm(embedded)
        return output, hidden

# Create the encoder with all the parameters
encoder = EncoderLSTM(embedding_dim=300, eng_vocab_size=eng_vocab_size, hidden_size=128, num_layers= 2, dropout_p=0.1).to(device)

In [12]:
encoder_output, encoder_output_hidden = encoder(encoder_input)

In [13]:
encoder_output.shape

torch.Size([10, 56, 128])

In [14]:
print(len(encoder_output_hidden))

print(encoder_output_hidden[0].shape, encoder_output_hidden[1].shape)

2
torch.Size([2, 10, 128]) torch.Size([2, 10, 128])


In [15]:
encoder_output.shape

torch.Size([10, 56, 128])

In [16]:
# Confirm that the output dimensions are what we expect:
print("output shape:", encoder_output.shape)
print("h_n output shape:", encoder_output_hidden[0].shape)
print("h_c output shape:", encoder_output_hidden[1].shape)

# the h_c output (context vector) layer is what goes into the decoder

# h_n is the last element in the output
# the 12th element of output == h_n
# but order is flipped (batch_first only on output, not on the hidden layers)

output shape: torch.Size([10, 56, 128])
h_n output shape: torch.Size([2, 10, 128])
h_c output shape: torch.Size([2, 10, 128])


In [23]:
# # dummy decoder input
guj_vocab_size = 25500 # based on vocab size above
# decoder_input = t.tensor(t.zeros(batch_size, guj_vocab_size)).long() 


In [24]:
class DecoderLSTM(nn.Module):
    def __init__(self, embedding_dim, guj_vocab_size, hidden_size, num_layers, dropout_p=0.1):
        """
        embedding_dim: for each word in guj_vocab_size, this is the number of floats representing each word
        guj_vocab_size: the size of the original gujarati vocab len(gu_words)
        hidden_size: the number of LSTM cells in each layer
        num_layers: the number of LSTM layers in the model
        dropout_p: 
        """
        super(DecoderLSTM, self).__init__()
        # Step 1: Create the Embedding Matrix
        # input dimensions: tensor[batch_size, sequence_length]
        # output dimensions: tensor[batch_size, sentence_length, embedding_dim]
        # creates the embedding matrix to be of dimensions vocab size x embedding_dim
        # the actual values are random to begin with
        # the embedding matrix is unique (lookup table, key is the index, value is the embedding for that word)
        self.embedding = nn.Embedding(guj_vocab_size, embedding_dim)
        # Step 2: Run the LSTM Model
        # In: input, (h_n, c_n)
        # input dimensions: (for batched) tensor[batch_size, sequence_length, input_size] AKA tensor[batch_size, sentence_length, embedding_dim]
        # h_n input dimensions: tensor[#_layers, hidden_size] of zeros since we don't specify
        # h_c input dimensions: tensor[#_layers, batch_size, hidden_size] of zeros since we don't specify
        # Out: output, (h_n, c_n)
        # output dimensions: (for batched) tensor[batch_size, sequence_length, hidden_size] AKA tensor[batch_size, sentence_length, number_of_LSTMcells]
        # h_n output dimensions: this is the hidden output (short term memory). tensor[#_layers, batch_size, hidden_size]
        # h_c output dimensions: this is the cell output (long term memory). tensor[#_layers, batch_size, hidden_size]
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout_p, batch_first=True)
        # Step 3: Generate the predicted word. This will give us the next word, and will also be used as a hidden layer for the next prediction
        # This is done by performing a linear transormation on 
        # In: any # of dimensions (including none), in_features
        # Out
        self.generator = nn.Linear(in_features=hidden_size, out_features=guj_vocab_size) 

    def forward(self, encoder_hidden: t.Tensor):
        # Big picture: 
        # The decoder takes in the hidden layer from the encoder, as well as the previous predicted word
        # For the first round, the previous predicted word is the start token
        # encoder_hidden # of size [batch_size, sequence_length, d_model_encoder] 
        batch_size = encoder_hidden.shape[0] # take the first element (batch size) of the second hidden layer (context vector)
        # why do we do this?? below
        decoder_hidden = encoder_hidden # value from encoder
        decoder_input_int = guj_word2index["<s>"] #  Get the index of the start token. this value is 1
        # goal is to get decoder_input to look like this
        # decoder_input * batch_size [1, 1, 1, 1] (len of batch size)
        # decoder_input should be a tensor of dimension [batch_size]
        decoder_input = t.tensor(t.ones(batch_size)*decoder_input_int).long()
        # forcing the batch size here so it must be 10. This is we why drop 1 seq

        # can feed in multiple sentences at a time (output is a list of tensors)
        decoder_outputs: list[t.Tensor] = [] # this is a list of 2D tensors. This is why we need stack later to force 3D
        # Our longest gujarati sentence is 46 tokens, so we will predict the next word up tp 50 times. 
        # This limit is in place to prevent our model from running infinitely
        for n in range(50):
            output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
            # output is batch size x vocab size
            # but we just need the index of the token (like the original input)
            # this is the predicted token that will be fed into the next step of the model run
            # can do this with the argmax
            # TODO: implement beam search 
            decoder_input = t.argmax(output, dim=-1).detach() # taking the max prob (explicitly greedy search!) argmax over vocab size!
            # detach here so that we don't compute gradients on this decoder_input
            decoder_outputs.append(output) # we want to save all of the predicted words we get along the way. Why??

        decoder_outputs = t.stack(decoder_outputs, dim=1) #concatenate into tensor of tensors; [batch_size, seq_len, guj_vocab_size], adds another dimension
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1) # softmax all the tensors at one time, over the guj_vocab_size dimension
        return decoder_outputs, decoder_hidden # We return `None` for consistency in the training loop????
    
    def forward_step(self, previous_output, hidden):
        """
        Conducts a forward step through one LSTM cell in the decoder.

        Returns:
            output (Tensor[batch_size, guj_vocab_size])
            hidden (Tensor[batch_size, hidden_size])
        """

        output = self.embedding(previous_output)
        output, hidden = self.lstm(output, hidden)
        # output = F.relu(output)
        output = self.generator(output)
        return output, hidden
    
# Create the decoder with all the parameters
decoder = DecoderLSTM(embedding_dim=300, guj_vocab_size=guj_vocab_size, hidden_size=128, num_layers= 2, dropout_p=0.1).to(device)


In [ ]:
# how many times do I want to do a forward step (how many words do I want to predict?)
# 50 (len of longest sentence is 46 tokens)

In [40]:
def training_step(batch, encoder, encoder_optimizer, decoder, decoder_optimizer, criterion):
    '''
    '''
    input_tensor, target_tensor = batch
    encoder_output, encoder_output_hidden = encoder(input_tensor)
    encoder_out = t.transpose(encoder_output_hidden[1], 0, 1)
    output, hidden = decoder(encoder_out)
    loss = criterion(
        output.view(-1, output.size(-1)), target_tensor.view(-1)
    )
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss

In [41]:
def validation_step(batch, encoder, decoder):
        '''
        Calculates & returns the accuracy on the tokens in the batch (i.e. how often the model's prediction
        is correct). Logging should happen in the `train` function (after we've computed the accuracy for
        the whole validation set).
        '''
        input_tensor, target_tensor = batch
        encoder_output, encoder_output_hidden = encoder(input_tensor)
        encoder_out = t.transpose(encoder_output_hidden[1], 0, 1)
        output, hidden = decoder(encoder_out)

        predictions = output.argmax(dim=-1)
        correct = (predictions == target_tensor).flatten()
        return correct

In [42]:
from tqdm import tqdm 
def train(train_dataloader, val_dataloader, encoder, decoder, n_epochs, learning_rate=0.001):
        # #print_every=100, plot_every=100):
        
        # # my old code
        # start = time.time()
        # plot_losses = []
        # print_loss_total = 0  # Reset every print_every
        # plot_loss_total = 0  # Reset every plot_every

        # Fern's plot code
        accuracy = np.nan

        progress_bar = tqdm(total = len(train_dataloader) * n_epochs)

        loss_logs = []
        loss_fig = plt.figure()
        loss_ax = plt.gca()
        loss_display = display(loss_fig, display_id=True)
        
        acc_logs = []
        acc_fig = plt.figure()
        acc_ax = plt.gca()
        acc_display = display(acc_fig, display_id=True) 

        encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
        criterion = nn.NLLLoss() #negative log likelihood loss

        for epoch in range(0, n_epochs):
            step = 0
            for i, batch in enumerate(train_dataloader):
                step += 1
                loss = training_step(batch, encoder, encoder_optimizer, decoder, decoder_optimizer, criterion)
                progress_bar.update()
                progress_bar.set_description(f"Epoch {epoch+1}, loss: {loss:.3f}, accuracy: {accuracy:.2f}")

                loss_logs.append(np.array(object=loss.detach().to(device)))
            
                if step % 100 == 0:
                    loss_ax.clear()
                    loss_ax.plot(loss_logs[100:])
                    loss_fig.canvas.draw()
                    loss_display.update(obj=loss_fig)

            correct_predictions = t.concat([validation_step(batch, encoder, decoder) for batch in val_dataloader])
            accuracy = correct_predictions.detach().to(device).float().mean().item()

            acc_logs.append(np.array(object=accuracy))
            acc_ax.clear()
            acc_ax.plot(acc_logs)
            acc_fig.canvas.draw()
            acc_display.update(obj=acc_fig)

            # proj_directory = "/Users/mkshah605/Documents/GitHub/RC/NMT_IndicLang/"

            # loss_fig.savefig(proj_directory + "images/loss_fig.png")
            # acc_fig.savefig(proj_directory + "images/acc_fig.png")
        
        return loss_logs, acc_logs

In [43]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, random_split


data = TensorDataset(t.LongTensor(eng_seq_collection[:-1]).to(device),
                               t.LongTensor(guj_seq_collection[:-1]).to(device))


train_dataset, val_dataset = random_split(data, [0.7, 0.3])


# Set up Training & Validation DataLoaders
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

val_sampler = RandomSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)

# Run Training Loop
train(train_dataloader, val_dataloader, encoder, decoder, n_epochs=1)

  0%|          | 0/4550 [00:00<?, ?it/s]

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

/var/folders/ww/cqrdw531507g4cfv8ncd8fz80000gp/T/ipykernel_1216/2293439555.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = t.tensor(t.ones(batch_size)*decoder_input_int).long()
Epoch 1, loss: 7.174, accuracy: nan: 100%|██████████| 4550/4550 [1:08:10<00:00,  1.11it/s] 


([array(10.170965, dtype=float32),
  array(10.12511, dtype=float32),
  array(10.07653, dtype=float32),
  array(10.047723, dtype=float32),
  array(10.020679, dtype=float32),
  array(9.922576, dtype=float32),
  array(9.743904, dtype=float32),
  array(9.507356, dtype=float32),
  array(9.214358, dtype=float32),
  array(8.903859, dtype=float32),
  array(8.560115, dtype=float32),
  array(8.128089, dtype=float32),
  array(7.7504134, dtype=float32),
  array(7.360582, dtype=float32),
  array(7.0671844, dtype=float32),
  array(6.645515, dtype=float32),
  array(6.246201, dtype=float32),
  array(5.878709, dtype=float32),
  array(5.5537934, dtype=float32),
  array(5.211566, dtype=float32),
  array(4.9281425, dtype=float32),
  array(4.720228, dtype=float32),
  array(4.291346, dtype=float32),
  array(3.9300354, dtype=float32),
  array(3.9452662, dtype=float32),
  array(3.3112128, dtype=float32),
  array(3.2309163, dtype=float32),
  array(2.927772, dtype=float32),
  array(2.720133, dtype=float32),
  a

In [ ]:
# Generating the output. The token can generate many different tokens.
# Generator stage: hidden tensor to vocab tensor, and then softmax to get probablitiy
# Greedy search is saying we want the highest probability token to be the word

# v.s., the greedy search at that time might not be the best at predicting the right word
# 

In [25]:
# # Capture the output in 2 different objects
# # note: encoder_output_hidden[1] is context vector
# # output, hidden = decoder(encoder_output_hidden[1][-1].unsqueeze(0)) # the -1 is because we need the output from the 2nd LSTM layer
# # the unsqueeze adds a dummy index (adds one more dimension)
# # unsqueeze --> [seq_length, batch_size]

# # need to transpose bc batch size needs to be first
# # transpose --> [batch_size, seq_len]
# # encoder_out shape: [batch_size, num_layers, hidden_size]
# encoder_out = t.transpose(encoder_output_hidden[1], 0, 1)
# output, hidden = decoder(encoder_out)


# # hidden output (short term memory). tensor[#_layers, batch_size, hidden_size]


/var/folders/ww/cqrdw531507g4cfv8ncd8fz80000gp/T/ipykernel_1216/2293439555.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = t.tensor(t.ones(batch_size)*decoder_input_int).long()


In [26]:
# # Prep the real data
# encoder_input = t.tensor(eng_seq_collection)
# len(encoder_input)

# # split the encoder input into batches
# i = 0
# j = 10
# encoder_input_list = []
# while j <= 65002:
#     encoder_input_list.append(encoder_input[i:j])
#     i = j
#     j += 10


# # Now we have 6500 tensors with dimensions batch_size, seq_len
# print(encoder_input_list[0].shape)
# print(len(encoder_input_list))

torch.Size([10, 55])
6500


In [27]:
# Training Run
# Capture the output in 2 different objects

# encoder_output_total = t.empty(10, 56, 128)
# encoder_output_hidden_total = t.cat((t.empty(2, 10, 128), t.empty(2, 10, 128)))
for encoder_input_batch in encoder_input_list[:1]:
    encoder_output, encoder_output_hidden = encoder(encoder_input_batch)
    encoder_out = t.transpose(encoder_output_hidden[1], 0, 1)
    output, hidden = decoder(encoder_out)
    predictions = output.argmax(dim=-1)
    x = map(lambda x: guj_index2word[x], predictions[0].tolist())
    y = map(lambda x: eng_index2word[x], encoder_input_batch[0].detach().tolist())
    print(list(y))
    print(list(x))
    break
    # encoder_output_total = t.stack((encoder_output_total, encoder_output))
    # encoder_output_hidden_total = t.stack((encoder_output_hidden_total, encoder_output_hidden))
    # encoder_output_list.append(encoder_output)
    # encoder_output_hidden_list.append(encoder_output_hidden)

['<s>', 'A', 'bicycle', 'replica', 'with', 'a', 'clock', 'as', 'the', 'front', 'wheel.', '</s>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>']
['બિંબ', 'બ્લેડ', 'દ્રશ્યની', 'વસ્તુની', 'મિશ્રણના', 'ગયા', 'પ્લેટફોર્મ્સની', 'બદલ્યો.', 'મિશ્રણના', 'ગયા', 'પીળાં', 'sleek', 'ગ્રેશ', 'હેન્ડલને', 'રહેણાંક', 'સ્કેટબોર્ડર્સ', 'બિલ્ટ-ઇન', 'ગયા', 'પંક્તિમાં', 'ગયા', 'એકલા', 'દીવાદાંડીના', 'આંતરડા', 'થયેલ.', 'રસોઇ,', 'બેસપૅક', 'ટોનથી', 'countertop.', 'ગિરરફેસનું', 'બૂટ્સમાં', 'એવન્યુ"', 'દર્શાવવામાં', 'વાડોમાં', 'સ્ટેજીંગ', 'પેસેન્જર્સને', 'તિરાડ', 'ગ્રેફિટીમાં', 'મિશ્રણના', 'ગયા', 'વાદળાંઓના', 'ઢાંકતી', 'ઝાડવુંની', 'છોડન

/var/folders/ww/cqrdw531507g4cfv8ncd8fz80000gp/T/ipykernel_1216/2293439555.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = t.tensor(t.ones(batch_size)*decoder_input_int).long()


In [28]:
# Training Run
# Capture the output in 2 different objects

# encoder_output_total = t.empty(10, 56, 128)
# encoder_output_hidden_total = t.cat((t.empty(2, 10, 128), t.empty(2, 10, 128)))
for encoder_input_batch in encoder_input_list[:1]:
    encoder_output, encoder_output_hidden = encoder(encoder_input_batch)
    encoder_out = t.transpose(encoder_output_hidden[1], 0, 1)
    output, hidden = decoder(encoder_out)
    predictions = output.argmax(dim=-1)
    x = map(lambda x: guj_index2word[x], predictions[0].tolist())
    y = map(lambda x: eng_index2word[x], encoder_input_batch[0].detach().tolist())
    print(list(y))
    print(list(x))
    break
    # encoder_output_total = t.stack((encoder_output_total, encoder_output))
    # encoder_output_hidden_total = t.stack((encoder_output_hidden_total, encoder_output_hidden))
    # encoder_output_list.append(encoder_output)
    # encoder_output_hidden_list.append(encoder_output_hidden)

['<s>', 'A', 'bicycle', 'replica', 'with', 'a', 'clock', 'as', 'the', 'front', 'wheel.', '</s>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>']
['બિંબ', 'બ્લેડ', 'બલૂન.', 'ડિસ્ટ્રિક્ટ', 'હાથીઓને', 'ઝાપટિયું', '"ફેક્ટરી', 'ગયા', 'બદલ્યો.', 'ટ્રોલી', 'મશરૂમથી', 'ગયા', 'થયેલ.', 'બલૂન.', '"ટિપ્સ', '"ફેક્ટરી', 'થયેલ.', 'મેટલની', 'મિશ્રણના', 'પવનમ', 'ડ્રોપ્સ', 'તિરાડ', 'બદલ્યો.', 'મિશ્રણના', 'ગયા', 'તિરાડ', 'બદલ્યો.', 'મિશ્રણના', 'પહોળાઇને', 'તિરાડ', 'ગયા', 'બેર', 'વાડોમાં', 'બફેટ', 'એન્જિનના', 'મોટા', 'વાડોમાં', 'ક્ષેત્રને', 'કૂદકા.', 'એડ્સ', 'વાડોમાં', 'કૂતરાના', 'બસ્સી', 'પહોળાઇને', 'સ્ટે', 'સપાટી,', 'સ્પાઘેટ્ટી'

/var/folders/ww/cqrdw531507g4cfv8ncd8fz80000gp/T/ipykernel_1216/2293439555.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = t.tensor(t.ones(batch_size)*decoder_input_int).long()


In [29]:
# Training Run
# Capture the output in 2 different objects

# encoder_output_total = t.empty(10, 56, 128)
# encoder_output_hidden_total = t.cat((t.empty(2, 10, 128), t.empty(2, 10, 128)))
for encoder_input_batch in encoder_input_list[:1]:
    encoder_output, encoder_output_hidden = encoder(encoder_input_batch)
    encoder_out = t.transpose(encoder_output_hidden[1], 0, 1)
    output, hidden = decoder(encoder_out)
    predictions = output.argmax(dim=-1)
    x = map(lambda x: guj_index2word[x], predictions[0].tolist())
    y = map(lambda x: eng_index2word[x], encoder_input_batch[0].detach().tolist())
    print(list(y))
    print(list(x))
    break
    # encoder_output_total = t.stack((encoder_output_total, encoder_output))
    # encoder_output_hidden_total = t.stack((encoder_output_hidden_total, encoder_output_hidden))
    # encoder_output_list.append(encoder_output)
    # encoder_output_hidden_list.append(encoder_output_hidden)

['<s>', 'A', 'bicycle', 'replica', 'with', 'a', 'clock', 'as', 'the', 'front', 'wheel.', '</s>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>']
['બિંબ', 'બ્લેડ', 'બ્રોકોલીનો', '"ફેક્ટરી', 'અકસ્માતમાં', 'લલામાસથી', 'ઈંટ', 'પ્રકારનો', 'પોચી', 'તિરાડ', 'બદલ્યો.', 'ક્લેર', 'સ્ટેટિક', 'ઉભરાઇ', 'ડાર્કમેકની', 'ગયા', 'પાકકળા', 'બારીઓ,', 'પ્રપોઝ', 'થયેલ.', 'ઢાંકેલો', 'ખબર', 'બદલ્યો.', 'મિશ્રણના', 'બેરલ', 'પ્લેટફોર્મ્સની', '"ખાવું', 'ઓક્ટોપસની', 'ટ્રેક્સનું', 'લાઇન', 'નિશાનવાળી', 'સ્ટ્રક્ચર', 'સિનના', 'ક્રેશ', 'પંક્તિમાં', 'શાળાના', 'કુડુસ', 'વાડોમાં', 'ગયા', 'પીળાં', 'રંગભેદ', 'મિશ્રણના', '"સ્ટોપ"', 'યાદોને', 'નળીમાંથી

/var/folders/ww/cqrdw531507g4cfv8ncd8fz80000gp/T/ipykernel_1216/2293439555.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = t.tensor(t.ones(batch_size)*decoder_input_int).long()


In [ ]:
# [[eng_tensor, gujtensor], [eng_tensor, gujtensor]]
# "dataloader" = list of tuples (eng, guj) as tokens

In [ ]:
# look at dataloader from transformers/GANs

In [30]:
# create a list of tuples with [(english, gujarati)]
sentence_pairs = []
for i in range(len(eng_seq_collection)):
    sentence_tuple = (eng_seq_collection[i], guj_seq_collection[i])
    sentence_pairs.append(sentence_tuple)

In [33]:
sentence_pairs[0][1]

[1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 25460,
 2]

In [34]:
guj_index2word[25460]

'</પેડ>'

In [ ]:
# TODO
# use randomsampler to split data into train and test (once it's in the paired tuples)
# establish dataloader from training and test data
# use code in the trasorer notebook (and solutions) to write train functions

In [31]:
x = map(lambda x: eng_index2word[x], sentence_pairs[0][0])
y = map(lambda x: guj_index2word[x], sentence_pairs[0][1])


print(list(x))
print(list(y))

['<s>', 'A', 'bicycle', 'replica', 'with', 'a', 'clock', 'as', 'the', 'front', 'wheel.', '</s>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>']
['<s>', 'ફ્રન્ટ', 'વ્હીલ', 'તરીકે', 'ઘડિયાળ', 'સાથે', 'સાયકલ', 'પ્રતિકૃતિ.', '</s>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>', '</પેડ>'

In [164]:
eng_index2word[17472]

'</pad>'

In [167]:
len(eng_seq_collection[6500])

55

In [ ]:
for word in eng_seq_collection[6500]:
    eng_index2word

In [32]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, random_split
data = TensorDataset(t.LongTensor(eng_seq_collection[:-1]).to(device),
                               t.LongTensor(guj_seq_collection[:-1]).to(device))

train_dataset, val_dataset = random_split(data, [0.7, 0.3])


# Set up Training & Validation DataLoaders
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

val_sampler = RandomSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=train_sampler, batch_size=batch_size)

In [60]:
for batch in train_dataloader:
    print(len(batch[0]))

50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
5

In [38]:
import random

random_element = random.choice(data)
data.drop(random_element)

AttributeError: 'TensorDataset' object has no attribute 'drop'

In [55]:
len(eng_seq_collection[:-1])

65000

In [52]:
#len(data[:-1][0])

print(data[:-1][0])

tensor([[    1,     3,     4,  ..., 17472, 17472, 17472],
        [    1,     3,    13,  ..., 17472, 17472, 17472],
        [    1,     3,    20,  ..., 17472, 17472, 17472],
        ...,
        [    1,     3,   461,  ..., 17472, 17472, 17472],
        [    1,     3,   731,  ..., 17472, 17472, 17472],
        [    1,     3,   157,  ..., 17472, 17472, 17472]])


In [53]:
#len(data)
print(data)

In [43]:
len(data)

65001

In [33]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [44]:
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [193]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    

In [45]:
# def get_dataloader(batch_size):
#     input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

#     n = len(pairs)
#     input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
#     target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

#     for idx, (inp, tgt) in enumerate(pairs):
#         inp_ids = indexesFromSentence(input_lang, inp)
#         tgt_ids = indexesFromSentence(output_lang, tgt)
#         inp_ids.append(EOS_token)
#         tgt_ids.append(EOS_token)
#         input_ids[idx, :len(inp_ids)] = inp_ids
#         target_ids[idx, :len(tgt_ids)] = tgt_ids

#     train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
#                                torch.LongTensor(target_ids).to(device))

#     train_sampler = RandomSampler(train_data)
#     train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
#     return input_lang, output_lang, train_dataloader

In [105]:
for encoder_input_batch in encoder_input_list[:1]:
    encoder_output, encoder_output_hidden = encoder(encoder_input_batch)
    encoder_out = t.transpose(encoder_output_hidden[1], 0, 1)
    output, hidden = decoder(encoder_out)
    predictions = output.argmax(dim=-1)
    x = map(lambda x: guj_index2word[x], predictions[0].tolist())
    y = map(lambda x: eng_index2word[x], encoder_input_batch[0].detach().tolist())
    print(list(y))
    print(list(x))

/var/folders/ww/cqrdw531507g4cfv8ncd8fz80000gp/T/ipykernel_98869/2013414216.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = t.tensor(t.ones(batch_size)*decoder_input_int).long()


['<s>', 'A', 'bicycle', 'replica', 'with', 'a', 'clock', 'as', 'the', 'front', 'wheel.', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</pad>', '</s>']
['ફળનો', 'આડ', 'કટિંગ', 'દંપતિ.', 'પ્રાઇઝ', 'દૂધનો', 'વાહનને', 'flying', 'પોટમાં', 'ખાય', 'દૂધનો', 'ટ્રે', 'ઝાકળવાળો', 'પેચની', 'બેરિન.', 'લાકડા-પેપર.', 'ગાજરની', '200', 'ટ્રેડેડ', 'આકાશગંગા', 'ઘરઆંગણે', 'juts.', 'જીરાફ.', 'શૉલ્ફ', 'અઢારમી', 'બ્લોઇંગ', 'ફલોર,', 'ગાજરની', 'સાધનસામગ્રી', 'દૂધનો', 'પ્રતિબિંબવાળી', 'સડેલા', 'દૂધનો', 'sleek', 'આયરન.', 'flying', 'સ્વાતંત્ર્યની', 'કાર્યકર્તા.', 'ભૂખરું', 'વધારી', 'સુયોજન', 'આયરન.', 'ડ્રેપેપાઇપ', 'સ્વાતંત્ર્યની', 'સે

In [176]:
from tqdm import tqdm
from tqdm import trange
from time import sleep
def train_epoch(epoch_number, dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    total_loss = 0
    for data in tqdm(dataloader, desc="Epoch Number {}".format(epoch_number)):

        input_tensor, target_tensor = data

        # if input_tensor.shape != t.Size([10, 55]):
        #     print(input_tensor.shape)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_output, encoder_output_hidden = encoder(input_tensor)
        encoder_out = t.transpose(encoder_output_hidden[1], 0, 1)

        # check size of all encoder_outputs
        # assert encoder_out.shape == t.Size([10, 2, 128])

        output, hidden = decoder(encoder_out)

        loss = criterion(
            output.view(-1, output.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        # print(loss)

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

        # See loss for each batch
        

    return total_loss / len(dataloader)

In [189]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss() #negative log likelihood loss

    #with tqdm(range(1, n_epochs + 1)) as epoch:
    # for epoch in tqdm(range(1, n_epochs + 1)):
    for epoch_number in range(1, n_epochs + 1):
        loss = train_epoch(epoch_number, train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch_number % 100 == 0:
            print_loss_avg = print_loss_total / 100
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch_number / n_epochs),
                                        epoch_number, epoch_number / n_epochs * 100, print_loss_avg))

        if epoch_number % 100 == 0:
            plot_loss_avg = plot_loss_total / 100
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #return plot_losses
    showPlot(plot_losses)

In [191]:
train(train_dataloader, encoder, decoder, n_epochs=1, print_every=1, plot_every=1)

Epoch Number 1:   0%|          | 0/20 [00:00<?, ?it/s]/var/folders/ww/cqrdw531507g4cfv8ncd8fz80000gp/T/ipykernel_98869/2293439555.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_input = t.tensor(t.ones(batch_size)*decoder_input_int).long()
Epoch Number 1: 100%|██████████| 20/20 [00:19<00:00,  1.01it/s]

0m 19s (- 0m 0s) (1 100%) 1.9910


In [21]:
for i in train_dataloader:
    print(len(i[0]))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

In [ ]:
def training_step(self, batch: Dict[str, Int[Tensor, "batch seq"]]) -> Float[Tensor, ""]:
    '''
    Calculates the loss on the tokens in the batch, performs a gradient update step, and logs the loss.

    Remember that `batch` is a dictionary with the single key 'tokens'.
    '''
    # YOUR CODE HERE
    tokens = batch["tokens"].to(device)
    output = self.model(tokens)
    loss = -get_log_probs(output, tokens).mean()
    loss.backward()
    self.optimizer.step()
    self.optimizer.zero_grad()
    return loss

def validation_step(self, batch: Dict[str, Int[Tensor, "batch seq"]]):
    '''
    Calculates & returns the accuracy on the tokens in the batch (i.e. how often the model's prediction
    is correct). Logging should happen in the `train` function (after we've computed the accuracy for
    the whole validation set).
    '''
    # YOUR CODE HERE
    tokens = batch["tokens"].to(device)
    output = self.model(tokens)[:,:-1]
    predictions = output.argmax(dim=-1)
    correct = (predictions == tokens[:,1:]).flatten()
    return correct

In [195]:
# good article: https://magazine.sebastianraschka.com/p/understanding-and-coding-self-attention

In [188]:
showPlot(plot_losses)

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, d_in, d_out_kq, d_out_v):
        # d_in: embedding size
        # d_out_qk:
        # d_out_v: 

        # note: keys and values usually come from encoder and queries come from decoder
        super().__init__()
        self.d_out_kq = d_out_kq
        self.W_query = nn.Parameter(t.rand(d_in, d_out_kq))
        self.W_key   = nn.Parameter(t.rand(d_in, d_out_kq))
        self.W_value = nn.Parameter(t.rand(d_in, d_out_v))

    def forward(self, x_1, x_2):
        # x_2 is new in cross attention
        # note: if we set x1 = x2, this is the same as self-attention
        # queries (decoder) are from x2 -> this is from gujarati in our case
        # keys and values (encoder) are from x1 -> from english
        # the attention mechanism is evaluating the interaction between two different inputs

        # each context vector is a weighted sum of the values.
        # But unlike self-attention, the values come from the 2nd input (x2) -> which comes from gujarati
        # The weights are based on the interaction between x1 and x2
        queries_1 = x_1 @ self.W_query
            
        keys_2 = x_2 @ self.W_key # new
        values_2 = x_2 @ self.W_value# new
            
        attn_scores = queries_1 @ keys_2.T # new 
        attn_weights = t.softmax(
            attn_scores / self.d_out_kq**0.5, dim=-1)
            
        context_vec = attn_weights @ values_2
        return context_vec

In [ ]:
# "test dataloader"

#

In [19]:
import torch
print(torch.backends.mps.is_available())

True
